# (Q)分子構造+(R)理由+(A)物性データセットのLLMによる学習と予測
- Q&A: 融点データセットを使用
- R: 自分自身で考えさせて､正解のデータを学習させる

In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from transformers import AutoTokenizer
import pandas as pd
import random
import copy
import glob
import json
from datetime import datetime
from llmchem.utils import mk_dir,clean_vram

#import clear_output

from IPython.display import clear_output

/home/setup/miniconda3/envs/chemllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#dataset settings
n_test=50 #number of testing data
n_train_check=50 #number of training data for checking (i.e., checking everything takes too long, so we check only a part of training data)
n_GPT_reasoning=10 # number of reasoning data made by GPT
n_generation_iterations=100   # trial numbers to generate new self reasoning data
max_generations=50

#model settings
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
target_modules= [
    "lm_head",
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate",
    "w1",
    "w2",
    "w3"
]

model_name=f"meta-llama/Llama-2-7b-chat-hf"
target_modules= [
    #"embed_tokens",
    "lm_head",
    #"q_proj",
    #"k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    #"down_proj",
]



#LoRA settings
r=32
lora_alpha=r
bit=16
#bit=8
#bit=4

#train settings
gradient_checkpointing =False
per_device_train_batch_size=1
epochs=3
lr=10**-5

#device settings
device_map="auto"

#dataset path
dataset_path="dataset/231225AutoReasoning/240117best_reason_record_3k.csv"

#project path
project_dir="results/projects/240117llama7b_10"

#reasoning options
error_threshold=30  # if abolute error is smaller than this, add to training data

In [3]:
mk_dir(project_dir)
mk_dir(project_dir+"/eval")
mk_dir(project_dir+"/self_reasoning")
mk_dir(project_dir+"/train")

In [4]:
from llmchem.model import init_model
from llmchem.train import train_model
from llmchem.eval import eval_model
from llmchem.reasoning import self_reasoning
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
#load base dataset

df=pd.read_csv(dataset_path)
dataset=df.to_dict(orient="records")
random.seed(0)
random.shuffle(dataset)

base_train_dataset=dataset[:-n_test]
example_reasoning_dataset=base_train_dataset[:n_GPT_reasoning]
test_dataset=dataset[-n_test:]

In [6]:
#Loop: training, evaluation, data generation
for generation in range(max_generations):
    clear_output()
    print(f"Generation: {generation}")
    #prepare train dataset

    ## reason data made by GPT4
    train_dataset=copy.deepcopy(example_reasoning_dataset)

    print(f"GPT-generated reasons: {len(train_dataset)}")

    ## reason data made by model itself
    for path in glob.glob(f"{project_dir}/self_reasoning/*.json"):
        with open(path) as f:
            train_dataset.append(json.load(f))

    print(f"All-generated reasons: {len(train_dataset)}")
    random.shuffle(train_dataset)

    #train model
    clean_vram()
    model=init_model(model_name, r, lora_alpha, target_modules, bit=bit,device_map=device_map)
    train_result=train_model(model,tokenizer,train_dataset,
                    project_dir=project_dir,
                    epochs=epochs,
                    lr=lr,
                    per_device_train_batch_size=per_device_train_batch_size,
                    gradient_checkpointing=gradient_checkpointing,
                    )

    #eval

    train_check_dataset=copy.deepcopy(train_dataset[:n_train_check])
    random.shuffle(train_check_dataset)
    train_eval_result=eval_model(model,tokenizer,train_check_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"train_{generation}"
                                )

    test_eval_result=eval_model(model,tokenizer,test_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"test_{generation}"
                                )

    #generate additional training data by self-reasoning
    self_reasoning(model,tokenizer,base_train_dataset,
                example_reasoning_dataset,project_dir,
                generation=generation,
                n_iterations=n_generation_iterations,
                error_threshold=error_threshold,
                n_max_trials=2)

Generation: 8
GPT-generated reasons: 10
All-generated reasons: 152
Using fp16 mode


Map: 100%|██████████| 152/152 [00:00<00:00, 10312.58 examples/s]


Step,Training Loss
100,1.439700
200,0.997600
300,0.790800
400,0.730000


  0%|          | 0/50 [00:00<?, ?it/s]

promlem 1 / 50


...

----


Heroin is a complex molecule with multiple functional groups and structural features that contribute to its melting point. The base compound, morphine, has a melting point of 105 degrees Celsius. The addition of a benzene ring and a methyl group increases the molecular weight and steric hindrance, which can contribute to a higher melting point. The nitro functional group can increase the melting point due to its electronegative nature and potential to form hydrogen bonds. The presence of a carbonyl group can also contribute to a higher melting point due to increased polarity and potential for hydrogen bonding. The methyl group can also contribute to a higher melting point due to its bulkiness and increased molecular weight.
##Prediction: 130.5


#Problem
actual:  173.0 predicted:  130.5
promlem 42 / 50


 84%|████████▍ | 42/50 [17:47<04:08, 31.03s/it]

----


The compound's melting point can be predicted by considering its functional groups:

- Benzene ring: The base unit benzene has a melting point of 5.5 degrees Celsius.
- Chlorine atom: Chlorine atoms increase the melting point due to increased molecular weight and polarity. Estimated contribution could be +50 degrees.
- Nitrile group: Nitriles typically raise the melting point due to hydrogen bonding and increased polarity. Estimated contribution could be +20 degrees.
- Amino group: Amines also raise the melting point due to strong hydrogen bonding and resonance stability. Estimated contribution could be +40 degrees.
- Cumulative impact: The individual contributions of each group can be summed up, but it's also necessary to consider that the presence of multiple polar and hydrogen bonding functional groups in one molecule can lead to synergistic effects, amplifying the increase in melting point beyond the sum of individual contributions.
Based on the above considerations, a rough

 86%|████████▌ | 43/50 [17:50<02:37, 22.45s/it]

----


##Prediction: -10.5


#Problem
actual:  -80.0 predicted:  -10.5
promlem 44 / 50
